In [ ]:
https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/mcp/agent_as_mcp_server.py

In [ ]:
import asyncio
from typing import Any

from agent_framework import AgentProtocol, AgentThread, HostedMCPTool
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

"""
Azure AI Agent with Hosted MCP Example

This sample demonstrates integration of Azure AI Agents with hosted Model Context Protocol (MCP)
servers, including user approval workflows for function call security.
"""


async def handle_approvals_with_thread(query: str, agent: "AgentProtocol", thread: "AgentThread"):
    """Here we let the thread deal with the previous responses, and we just rerun with the approval."""
    from agent_framework import ChatMessage

    result = await agent.run(query, thread=thread, store=True)
    while len(result.user_input_requests) > 0:
        new_input: list[Any] = []
        for user_input_needed in result.user_input_requests:
            print(
                f"User Input Request for function from {agent.name}: {user_input_needed.function_call.name}"
                f" with arguments: {user_input_needed.function_call.arguments}"
            )
            user_approval = input("Approve function call? (y/n): ")
            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(user_approval.lower() == "y")],
                )
            )
        result = await agent.run(new_input, thread=thread, store=True)
    return result


async def main() -> None:
    """Example showing Hosted MCP tools for a Azure AI Agent."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        # enable azure-ai observability
        await chat_client.setup_azure_ai_observability()
        agent = chat_client.create_agent(
            name="DocsAgent",
            instructions="You are a helpful assistant that can help with microsoft documentation questions.",
            tools=HostedMCPTool(
                name="Microsoft Learn MCP",
                url="https://learn.microsoft.com/api/mcp",
            ),
        )
        thread = agent.get_new_thread()
        # First query
        query1 = "How to create an Azure storage account using az cli?"
        print(f"User: {query1}")
        result1 = await handle_approvals_with_thread(query1, agent, thread)
        print(f"{agent.name}: {result1}\n")
        print("\n=======================================\n")
        # Second query
        query2 = "What is Microsoft Agent Framework?"
        print(f"User: {query2}")
        result2 = await handle_approvals_with_thread(query2, agent, thread)
        print(f"{agent.name}: {result2}\n")

await main()


[2025-10-13 09:48:03 - c:\Users\memasanz\repos\agent-framework-exploration\.venv\Lib\site-packages\azure\monitor\opentelemetry\exporter\statsbeat\_manager.py:189 - WARNING] Statsbeat re-initialized with the same configuration.
[2025-10-13 09:48:03 - c:\Users\memasanz\repos\agent-framework-exploration\.venv\Lib\site-packages\azure\monitor\opentelemetry\exporter\statsbeat\_manager.py:189 - WARNING] Statsbeat re-initialized with the same configuration.
[2025-10-13 09:48:03 - c:\Users\memasanz\repos\agent-framework-exploration\.venv\Lib\site-packages\azure\monitor\opentelemetry\exporter\statsbeat\_manager.py:189 - WARNING] Statsbeat re-initialized with the same configuration.


User: How to create an Azure storage account using az cli?
DocsAgent: To create an **Azure Storage account** using the **Azure CLI (`az cli`)**, follow these steps:

---

### 1. **Sign in to Azure**

```sh
az login
```

---

### 2. **Set Your Subscription (Optional, if multiple subscriptions)**

```sh
az account set --subscription "<your-subscription-id-or-name>"
```

---

### 3. **Create a Resource Group (if you don’t have one)**

```sh
az group create --name myResourceGroup --location eastus
```

---

### 4. **Create the Storage Account**

```sh
az storage account create \
  --name mystorageaccount123 \
  --resource-group myResourceGroup \
  --location eastus \
  --sku Standard_LRS \
  --kind StorageV2
```

**Parameters:**
- `--name`: globally unique name (3-24 lowercase letters/numbers, no special chars)
- `--resource-group`: name of your resource group
- `--location`: Azure region (e.g., eastus, westus, etc.)
- `--sku`: pricing tier (e.g., Standard_LRS, Standard_GRS, Premium_LRS, e